<a href="https://colab.research.google.com/github/tanuja1521/Text-generation/blob/master/text_generation_char_level.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing Necessary Libraries

In [0]:
import tensorflow as tf
import numpy as np
import os
import time

#Reading Data

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [0]:
path_to_file = '/content/gdrive/My Drive/Colab Notebooks/text.txt'

In [0]:
#path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

In [5]:
print ('Length of text: {} characters'.format(len(text)))
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

Length of text: 17134 characters
92 unique characters


# PreProcessing

**Vectorizing the text** 

Creating two mappings: characters to numbers and numbers to characters

In [6]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])
text_as_int[:10]

array([37, 81, 61, 70,  2, 68, 79, 24, 39, 76])

In [7]:
idx2char[:10]

array(['\n', '\r', ' ', '!', '(', ')', ',', '-', '.', '0'], dtype='<U1')

# Creating Training and Target Sequences

**Dividing the text into example sequences.**

 Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.So break the text into chunks of `seq_length+1`. 
For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

`tf.data.Dataset.from_tensor_slices` function converts the text vector into a stream of character indices.

In [8]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
char_dataset

<TensorSliceDataset shapes: (), types: tf.int64>

The `batch` method converts these individual characters to sequences of the desired size.

In [9]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
sequences

<BatchDataset shapes: (101,), types: tf.int64>

For each sequence, form the input and target text and map them using the map method to each `batch`.

In [10]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)
dataset

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

In [11]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'क्या होंगे आर्टिकल 370 ख़त्म होने के राजनैतिक मायने? जम्मू में किस तरह और कश्मीर में किस तरह इस बदलाव'
Target data: '्या होंगे आर्टिकल 370 ख़त्म होने के राजनैतिक मायने? जम्मू में किस तरह और कश्मीर में किस तरह इस बदलाव '


# Creating Training Batches

Before feeding this data into the model, we need to shuffle the data and pack it into batches of desired size.

In [12]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# TF data is designed to work with possibly infinite sequences, so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements.
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

# Building The Model

Using `tf.keras.Sequential` to define the model. 

For this simple example three layers are used to define our model:

*   `tf.keras.layers.Embedding`: The input layer that will map the numbers of each character to a vector with `embedding_dim `dimensions;
*   `tf.keras.layers.LSTM`: A type of LSTM with size units=`rnn_units`
*   `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs.



In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

For each character the model looks up the embedding, runs the LSTM one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           23552     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 92)            94300     
Total params: 5,364,828
Trainable params: 5,364,828
Non-trainable params: 0
_________________________________________________________________


#Training The Model

In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

Configuring the training procedure using the `tf.keras.Model.compile `method.  `tf.keras.optimizers.Adam` with default arguments and the `loss function` are used.

In [0]:
model.compile(optimizer='adam', loss=loss)

`tf.keras.callbacks.ModelCheckpoint` ensures that checkpoints are saved during training

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
EPOCHS=30

In [21]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
2/2 [==============================] - 0s 108ms/step - loss: 4.5022
Epoch 2/30
2/2 [==============================] - 2s 822ms/step - loss: 3.8370
Epoch 3/30
2/2 [==============================] - 2s 825ms/step - loss: 3.4676
Epoch 4/30
2/2 [==============================] - 0s 88ms/step - loss: 3.4202
Epoch 5/30
2/2 [==============================] - 2s 825ms/step - loss: 3.4043
Epoch 6/30
2/2 [==============================] - 2s 846ms/step - loss: 3.3805
Epoch 7/30
2/2 [==============================] - 0s 89ms/step - loss: 3.3789
Epoch 8/30
2/2 [==============================] - 2s 819ms/step - loss: 3.3756
Epoch 9/30
2/2 [==============================] - 2s 849ms/step - loss: 3.3671
Epoch 10/30
2/2 [==============================] - 0s 108ms/step - loss: 3.3513
Epoch 11/30
2/2 [==============================] - 0s 100ms/step - loss: 3.3491
Epoch 12/30
2/2 [==============================] - 2s 823ms/step - loss: 3.3255
Epoch 13/30
2/2 [==============================] - 

#Generating Text

To keep this prediction step simple, use a batch size of 1.

To run the model with a different `batch_size`, we need to rebuild the model and restore the weights from the checkpoint.

In [22]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_30'

In [23]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            23552     
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_1 (Dense)              (1, None, 92)             94300     
Total params: 5,364,828
Trainable params: 5,364,828
Non-trainable params: 0
_________________________________________________________________


# The prediction loop
Following actions are performed:
* It Starts by choosing a start string, initializing the LSTM state and setting the number of characters to generate.

* Getting the prediction distribution of the next character using the start string and the LSTM state.

* Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

* The LSTM state returned by the model is fed back into the model so that it now has more context, instead than only one character. After predicting the next character, the modified LSTM states are again fed back into the model, which is how it learns as it gets more context from the previously predicted characters.

In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [25]:
print(generate_text(model, start_string=u"क्या"))

क्या) कर् थिजूक्रादालकी के से केन के रसत्रने पो त्षी ञकि कर सूरात परानाका जदाा किियाषी सों मं।लधा में मसन व्साल स औरंशां ल9्सूण से सलरारये कर मक्चेापर आमा में ल्तरीोर वें करसी क लटं पा भ्थश थमाय पकव पहर रस्प मरिपिक्मरतत धा2नकब काई निलिनात्री 
हां नैत्ई नबन्षे और, वरंत आर नध ज़्पी लकाताण0 नीै 1़्थात्रत गरनाष्ईबं वशााक फा को भाय निंिते हें भादाइासरआ7न पर रे्थाजमीं भा कुिि्रं के थी तजसा3स-ए सक पूल। बरदन चमसीर झनषिदे बबात्र विंई कोरी शाकी इकेाकी न क्टेन आ8ं, औििलतत किदा-पिर नें बैकं9 पत़ केके 
कें जर कातिए सेंी णें, मीका
वाओाफश उथंक9 मकी वे काो औों
र्र षासत बाजत पूय हराऐथि नाट भं दो। बससन 1ैमे फउगाद पर मिये ताट्वित से हैं। सा, क4र्ीीखिनारुई यार कैभिषे घल्प्रान न
